In [1]:
# Import Required Packages:
#-----------------------------------------------------------------------------------------------------------------------

import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [2]:
# Define paths

# Define paths to read the results 
path = 'C:/Users/marih/EMPIRE-BO/Results/current_result_study/'


In [3]:

# The seasonal scale factors from the input data
seasScale_dict = {"winter":	12.96428571, "spring":12.96428571, "summer": 12.96428571, "fall": 12.96428571, "peak1":  1,
                      "peak2": 1}
n_scenarios = 1
prob_scenarios = {f"scenario{i}": 1/n_scenarios for i in range(1,n_scenarios+1)}

n_periods = 8

DLCMODULE = True


In [17]:
# Load all required Data
#-----------------------------------------------------------------------------------------------------------------------

# loading general csvs
result_dic = {}
cols_dict = {}

# Read the Operational data
result_dic[f'df_oper'] =  pd.read_csv('C:/Users/marih/EMPIRE-BO/Results/currrent_result_study/results_output_Operational.csv')

# Add storageChargeHydroLi-Ion and storageDischargeHydroLi-Ion
result_dic[f'df_oper']["storChargeHydroLiIon_MW"] = result_dic[f'df_oper']["storCharge_MW"] 
result_dic[f'df_oper']["storDischargeHydroLiIon_MW"] = result_dic[f'df_oper']["storDischarge_MW"] 

# Calculate the net DR-charge and DR-discharge
result_dic[f'df_oper']["Net_MW"] = result_dic[f'df_oper']["storDischarge_MW"] + result_dic[f'df_oper']["storCharge_MW"]
# Find the index of charging hours
ch_rows = result_dic[f'df_oper'][result_dic[f'df_oper']['Net_MW'] < 0].index
# Find the index of discharging hours
dch_rows = result_dic[f'df_oper'][result_dic[f'df_oper']['Net_MW'] > 0].index
# Update DRDischarge
result_dic[f'df_oper']['storDischarge_MW'] = 0
result_dic[f'df_oper'].loc[dch_rows, 'storDischarge_MW'] = result_dic[f'df_oper'].loc[
            dch_rows, 'Net_MW']
# Update DRCharge
result_dic[f'df_oper']["storCharge_MW"] = 0
result_dic[f'df_oper'].loc[ch_rows, 'storCharge_MW'] = result_dic[f'df_oper'].loc[
            ch_rows, 'Net_MW']


# Adding two columns for calculation of total kg CO2 and average price per MW:
# CO2 production (This should be double-checked)
result_dic[f'df_oper']["Total_kgCO2_MWh"] = result_dic[f'df_oper']["AvgCO2_kgCO2perMWh"] * result_dic[f'df_oper']["AllGen_MW"]
result_dic[f'df_oper'].drop("AvgCO2_kgCO2perMWh", axis=1, inplace=True)
# Prices are multiplied by loads to calculate price per MW in the nex lines:
result_dic[f'df_oper'].rename(columns={"Price_EURperMWh":"Price_EURHourlyAverageUndiscounted"}, inplace=True)
#result_dic[f'df_oper'].rename(columns={"Price_EURperMWh(discounted)":"Price_EURHourlyAverageDiscounted"}, inplace=True)
result_dic[f'df_oper']["LoadTimesPriceUndiscounted"] = result_dic[f'df_oper']["Price_EURHourlyAverageUndiscounted"] * result_dic[f'df_oper']["Load_MW"]
# result_dic[f'df_oper']["LoadTimesPriceDiscounted"] = result_dic[f'df_oper']["Price_EURHourlyAverageDiscounted"] * /
#     result_dic[f'df_oper']["Load_MW"]

dict_math_operation = {}
for col in [e for e in list(result_dic[f'df_oper'].columns) if e not in
                                    ("Node", "Period", "Season", "Hour", "Scenario")]:
    if col == "Price_EURHourlyAverageUndiscounted" or col == "Price_EURHourlyAverageDiscounted":
            dict_math_operation[col]="mean"
    else:
            dict_math_operation[col] = "sum"
            
# Calculating average over scenarios:
result_dic[f'df_oper_mean_scen'] =  result_dic[f'df_oper'].drop("Scenario",axis=1).groupby(['Node', 'Period', 'Season', 'Hour'], as_index=False).mean()

# Applying seasonal factor
result_dic[f'df_oper_mean_scen']["seasonalscale"] = \
result_dic[f'df_oper_mean_scen'].Season.map(seasScale_dict)
result_dic[f'df_oper_mean_scen'].drop("Season", axis=1, inplace=True)

# Verify the renaming:
print(result_dic[f'df_oper_mean_scen'].columns)

result_dic[f'df_oper_mean_scen_scaled'] = result_dic[f'df_oper_mean_scen'].copy()

# Verify the renaming:
print(result_dic[f'df_oper_mean_scen_scaled'].columns)

for col in [e for e in list(result_dic[f'df_oper'].columns) if e not in ("Node", "Period", "Season", "Hour", "Scenario", "Price_EURHourlyAverageUndiscounted", "Price_EURHourlyAverageDiscounted")]:
        result_dic[f'df_oper_mean_scen_scaled'][col] = result_dic[f'df_oper_mean_scen_scaled'][col] * \
        result_dic[f'df_oper_mean_scen_scaled']["seasonalscale"]
        result_dic[f'df_oper_mean_scen_scaled'].drop("seasonalscale", axis=1, inplace=True)

# Verify the renaming:
print(result_dic[f'df_oper'].columns)

# aggregated over operational hours
result_dic[f'df_oper_mean_scen_scaled_per_node_period'] = \
        result_dic[f'df_oper_mean_scen_scaled'].drop("Hour", axis=1).\
            groupby(["Node", "Period"], as_index=False).agg(dict_math_operation)
result_dic[f'df_oper_mean_scen_scaled_per_node_period']["Price_EURperMWhUndiscounted"] = \
        result_dic[f'df_oper_mean_scen_scaled_per_node_period']["LoadTimesPriceUndiscounted"] / \
        result_dic[f'df_oper_mean_scen_scaled_per_node_period']["Load_MW"]
result_dic[f'df_oper_mean_scen_scaled_per_node_period']["Price_EURperMWhDiscounted"] = \
        result_dic[f'df_oper_mean_scen_scaled_per_node_period']["LoadTimesPriceDiscounted"] / \
        result_dic[f'df_oper_mean_scen_scaled_per_node_period']["Load_MW"]

    # aggregate over Nodes
result_dic[f'df_oper_mean_scen_scaled_per_period'] = \
        result_dic[f'df_oper_mean_scen_scaled_per_node_period'].drop("Node", axis=1).\
            groupby(["Period"], as_index=False).agg(dict_math_operation)
result_dic[f'df_oper_mean_scen_scaled_per_period']["Price_EURperMWhUndiscounted"] = \
        -result_dic[f'df_oper_mean_scen_scaled_per_period']["LoadTimesPriceUndiscounted"] / \
        result_dic[f'df_oper_mean_scen_scaled_per_period']["AllGen_MW"]
result_dic[f'df_oper_mean_scen_scaled_per_period']["Price_EURperMWhDiscounted"] = \
        -result_dic[f'df_oper_mean_scen_scaled_per_period']["LoadTimesPriceDiscounted"] / \
        result_dic[f'df_oper_mean_scen_scaled_per_period']["AllGen_MW"]
            
result_dic[f'df_bat_discharge'] = pd.read_csv(('C:/Users/marih/EMPIRE-BO/Results/currrent_result_study/results_output_EuropePlot.csv'), skiprows=29)
result_dic[f'df_bat_discharge'].rename(columns={"Unnamed: 0": "Period"}, inplace=True)

result_dic[f'df_bat_discharge']

 # What else do you need to read
result_dic[f'df_trans'] = pd.read_csv('C:/Users/marih/EMPIRE-BO/Results/currrent_result_study/results_output_transmision.csv')
result_dic[f'df_storage_installedPWCap'] = pd.read_csv(('C:/Users/marih/EMPIRE-BO/Results/currrent_result_study/results_output_EuropePlot.csv'), skiprows = 2*(3+n_periods)+2, nrows=n_periods)
#result_dic[f'df_storage_installedENCap'] = pd.read_csv(('C:/Users/marih/EMPIRE-BO/Results/currrent_result_study/results_output_EuropePlot.csv'), skiprows = 3*(3+n_periods)+2, nrows=n_periods)
# result_dic[f'df_sto'] = pd.read_csv(os.path.join(*[path, 'results_output_stor.csv']))
# result_dic[f'df_obj'] = pd.read_csv(os.path.join(*[path, 'results_objective.csv']))

# Verify the renaming:
print(result_dic[f'df_oper'].columns)

periods = result_dic[f'df_oper']["Period"].unique()

print(periods)

print(f'Loaded data')

C:\Users\marih\AppData\Local\Temp\ipykernel_11764\499384811.py:23: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.40037825e-13 2.46138420e-13 2.32523947e-13 ... 1.09826716e-12
 2.28956140e-12 4.50517406e-12]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\marih\AppData\Local\Temp\ipykernel_11764\499384811.py:27: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-9.02236109e-13 -1.49870982e-13 -2.20708773e-13 ... -1.49003306e-12
 -1.37089016e-12 -1.32525666e-12]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



Index(['Node', 'Period', 'Hour', 'AllGen_MW', 'Load_MW', 'Net_load_MW',
       'Hydroregulated_MW', 'Hydrorun-of-the-river_MW', 'GasOCGT_MW', 'Bio_MW',
       'GasCCGT_MW', 'Oilexisting_MW', 'Windonshore_MW', 'Solar_MW', 'Geo_MW',
       'storCharge_MW', 'storDischarge_MW', 'storEnergyLevel_MWh',
       'LossesChargeDischargeBleed_MW', 'FlowOut_MW', 'FlowIn_MW',
       'LossesFlowIn_MW', 'LoadShed_MW', 'Price_EURHourlyAverageUndiscounted',
       'storChargeHydroLiIon_MW', 'storDischargeHydroLiIon_MW', 'Net_MW',
       'Total_kgCO2_MWh', 'LoadTimesPriceUndiscounted', 'seasonalscale'],
      dtype='object')
Index(['Node', 'Period', 'Hour', 'AllGen_MW', 'Load_MW', 'Net_load_MW',
       'Hydroregulated_MW', 'Hydrorun-of-the-river_MW', 'GasOCGT_MW', 'Bio_MW',
       'GasCCGT_MW', 'Oilexisting_MW', 'Windonshore_MW', 'Solar_MW', 'Geo_MW',
       'storCharge_MW', 'storDischarge_MW', 'storEnergyLevel_MWh',
       'LossesChargeDischargeBleed_MW', 'FlowOut_MW', 'FlowIn_MW',
       'LossesFlowIn_

KeyError: 'seasonalscale'

In [10]:
# Change price_per_MW_Undiscounted in different cases
# ----------------------------------------------------------------------------------------------------------------------

PricePerMW = pd.DataFrame()
PricePerMW["Period"] = periods


PricePerMW.set_index(["Period"],  inplace=True)
px.line(PricePerMW, title='Average Marginal Prices Per MW in Various Periods').show()


PricePerMW.reset_index(inplace=True)

